In [92]:
import copy
import pickle
import numpy as np
import networkx as nx
from itertools import product
from collections import OrderedDict

In [2]:
lambda_rate_sp = [2, 4, 6, 8, 10]
cost_diff_sp = [1.2, 1.4, 1.6, 1.8, 2.0]
func_num_sp = [10, 20, 30, 40, 50]
cpu_core_sp = [100, 200, 300, 400, 500]
cache_capacity_sp = [256, 512, 768, 1024]
soft_ddl_param_sp = [[0.1, 0.2], [0.2, 0.3], [0.3, 0.4], [0.4, 0.5]]

node_selection_strategy_sp = [
    "NS_min_dis_own",
    "NS_min_dis_all",
    "NS_min_user",
    "NS_max_node",
    "NS_exist_func",
]
path_selection_strategy_sp = ["PS_min_dis", "PS_max_cache", "PS_max_cpu"]
task_sorting_strategy_sp = [
    "TS_exec_time_asc",
    "TS_data_vol_asc",
    "TS_exec_time_to_data_vol_ratio_asc",
    "TS_closest_soft_ddl",
    "TS_highest_response_ratio",
]

combinations = list(
    product(lambda_rate_sp, cost_diff_sp, func_num_sp, cpu_core_sp,
            cache_capacity_sp, soft_ddl_param_sp, node_selection_strategy_sp,
            path_selection_strategy_sp, task_sorting_strategy_sp))

In [3]:
len(combinations)

750000

In [4]:
with open(r'outData/comm.pkl', 'wb') as file:
    pickle.dump(combinations, file)

In [10]:
# user.pkl
# 用户Id -> [
#            内网拓扑最近的边缘节点,
#            全网拓扑最近的边缘节点,
#            覆盖用户数最少的边缘节点,
#            邻居节点数最多的边缘节点,
#            以距离远近排序的边缘节点列表
#           ]

In [17]:
with open(r"outData/user.pkl", "rb") as file:
    user = pickle.load(file)
with open(r"outData/node.pkl", "rb") as file:
    node = pickle.load(file)

In [12]:
user[0]

array([[ 52,   1, 369],
       [ 93,   1, 146],
       [ 53,   1, 835],
       [ 12,   1, 222],
       [116,   1, 629],
       [ 50,   1, 549],
       [ 86,   1, 378],
       [ 22,   1, 902],
       [108,   6, 269],
       [  0,   6,  67],
       [ 92,   6,  63],
       [115,   6, 356],
       [ 21,   6, 430],
       [ 23,  24, 209],
       [ 87,  24, 147],
       [ 27,  24, 246],
       [ 34,  24, 167],
       [109,  24, 246]])

In [13]:
len(user)

816

In [18]:
##################################
# ---------节点选择策略---------- #
##################################
# 1.min_dis_own:选择距离最近的本服务提供商的基站
def NS_min_dis_own(userId, funcId=-1):
    serviceId = 1 if userId % 3 == 1 else (6 if userId % 3 == 2 else 24)
    sortedNdLst = sorted(user[userId], key=lambda x: x[2])
    for nd in sortedNdLst:
        if nd[1] == serviceId:
            return nd[0]

# 2.min_dis_all:选择距离最近的基站
def NS_min_dis_all(userId, funcId=-1):
    return min(user[userId], key=lambda x: x[2])[0]

# 3.min_user:选择覆盖用户数最少的基站
def NS_min_user(userId, funcId=-1):
    return min(user[userId], key=lambda x: node[x[0]][0])[0]

# 4.max_node:选择邻居基站数最多的基站
def NS_max_node(userId, funcId=-1):
    return max(user[userId], key=lambda x: node[x[0]][1])[0]

In [26]:
newUser={}
for userId in user:
    newUser[userId] = [NS_min_dis_own(userId), NS_min_dis_all(userId), NS_min_user(userId), NS_max_node(userId), [x[0] for x in sorted(user[userId], key=lambda x: x[2])]]

In [27]:
newUser[0]

[87,
 92,
 87,
 53,
 [92, 0, 93, 87, 34, 23, 12, 27, 109, 108, 115, 52, 86, 21, 50, 116, 53, 22]]

In [31]:
with open(r'outData/user-2.pkl', 'wb') as file:
    pickle.dump(newUser, file)

In [34]:
len(newUser)

816

In [110]:
with open(r"outData/graph-1.pkl", "rb") as file:
    graph = pickle.load(file)

In [111]:
graph

{'sub-1': <networkx.classes.digraph.DiGraph at 0x7f9bb410ff90>,
 'sub-6': <networkx.classes.digraph.DiGraph at 0x7f9b72c2af10>,
 'mmWave-24': <networkx.classes.digraph.DiGraph at 0x7f9b72c589d0>,
 'cross-net': <networkx.classes.digraph.DiGraph at 0x7f9b73085610>}

In [112]:
G_sub_1 = copy.deepcopy(graph["sub-1"])
G_sub_6 = copy.deepcopy(graph["sub-6"])
G_mmWave_24 = copy.deepcopy(graph["mmWave-24"])

In [113]:
for u, v, _ in G_sub_1.edges(data=True):
    G_sub_1[u][v]["weight"] = 1.0 / 1000.0

for u, v, _ in G_sub_6.edges(data=True):
    G_sub_6[u][v]["weight"] = 1.0 / 2000.0

for u, v, _ in G_mmWave_24.edges(data=True):
    G_mmWave_24[u][v]["weight"] = 1.0 / 4000.0

In [114]:
adjDict = {}
G_1 = {}
G_6 = {}
G_24 = {}
for nd in G_sub_1.nodes():
    G_1[nd] = dict(nx.shortest_path_length(G_sub_1, nd, weight="weight"))
for nd in G_sub_6.nodes():
    G_6[nd] = dict(nx.shortest_path_length(G_sub_6, nd, weight="weight"))
for nd in G_mmWave_24.nodes():
    G_24[nd] = dict(nx.shortest_path_length(G_mmWave_24, nd, weight="weight"))
adjDict[1] = G_1
adjDict[6] = G_6
adjDict[24] = G_24

In [115]:
sub_1 = np.array(
    [30, 89, 52, 57, 93, 53, 1, 56, 12, 116, 38, 50, 86, 76, 84, 28, 22, 98])
sub_6 = np.array([
    103, 54, 108, 45, 95, 36, 58, 14, 6, 11, 0, 96, 88, 25, 104, 113, 80, 81,
    94, 7, 31, 92, 115, 39, 21, 72, 4, 9, 105, 85, 37, 63, 107, 17, 79, 26
])
mmWave_24 = np.array([
    32, 55, 124, 118, 101, 3, 106, 97, 66, 19, 68, 13, 122, 70, 23, 59, 15, 47,
    10, 87, 102, 65, 29, 2, 73, 5, 75, 114, 117, 77, 33, 35, 60, 8, 46, 111,
    112, 71, 43, 61, 27, 51, 69, 121, 90, 16, 41, 64, 67, 34, 99, 120, 123, 62,
    100, 74, 109, 42, 119, 48, 78, 24, 83, 110, 20, 44, 91, 40, 82, 18, 49
])
cost_diff = 1.2
G_cross_net = copy.deepcopy(graph["cross-net"])
for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = 1000.0
    elif u in sub_6:
        bandwidth = 2000.0
    else:
        bandwidth = 4000.0
    if ((u in sub_1 and v in sub_1) or (u in sub_6 and v in sub_6)
            or (u in mmWave_24 and v in mmWave_24)):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth
G_tmp = {}
for nd in G_cross_net.nodes():
    G_tmp[nd] = dict(nx.shortest_path_length(G_cross_net, nd, weight="weight"))
adjDict[cost_diff] = G_tmp

In [116]:
cost_diff = 1.4
G_cross_net = copy.deepcopy(graph["cross-net"])
for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = 1000.0
    elif u in sub_6:
        bandwidth = 2000.0
    else:
        bandwidth = 4000.0
    if ((u in sub_1 and v in sub_1) or (u in sub_6 and v in sub_6)
            or (u in mmWave_24 and v in mmWave_24)):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth
G_tmp = {}
for nd in G_cross_net.nodes():
    G_tmp[nd] = dict(nx.shortest_path_length(G_cross_net, nd, weight="weight"))
adjDict[cost_diff] = G_tmp

In [117]:
cost_diff = 1.6
G_cross_net = copy.deepcopy(graph["cross-net"])
for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = 1000.0
    elif u in sub_6:
        bandwidth = 2000.0
    else:
        bandwidth = 4000.0
    if ((u in sub_1 and v in sub_1) or (u in sub_6 and v in sub_6)
            or (u in mmWave_24 and v in mmWave_24)):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth
G_tmp = {}
for nd in G_cross_net.nodes():
    G_tmp[nd] = dict(nx.shortest_path_length(G_cross_net, nd, weight="weight"))
adjDict[cost_diff] = G_tmp

In [118]:
cost_diff = 1.8
G_cross_net = copy.deepcopy(graph["cross-net"])
for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = 1000.0
    elif u in sub_6:
        bandwidth = 2000.0
    else:
        bandwidth = 4000.0
    if ((u in sub_1 and v in sub_1) or (u in sub_6 and v in sub_6)
            or (u in mmWave_24 and v in mmWave_24)):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth
G_tmp = {}
for nd in G_cross_net.nodes():
    G_tmp[nd] = dict(nx.shortest_path_length(G_cross_net, nd, weight="weight"))
adjDict[cost_diff] = G_tmp

In [119]:
cost_diff = 2.0
G_cross_net = copy.deepcopy(graph["cross-net"])
for u, v, _ in G_cross_net.edges(data=True):
    if u in sub_1:
        bandwidth = 1000.0
    elif u in sub_6:
        bandwidth = 2000.0
    else:
        bandwidth = 4000.0
    if ((u in sub_1 and v in sub_1) or (u in sub_6 and v in sub_6)
            or (u in mmWave_24 and v in mmWave_24)):
        G_cross_net[u][v]["weight"] = 1.0 / bandwidth
    else:
        G_cross_net[u][v]["weight"] = cost_diff / bandwidth
G_tmp = {}
for nd in G_cross_net.nodes():
    G_tmp[nd] = dict(nx.shortest_path_length(G_cross_net, nd, weight="weight"))
adjDict[cost_diff] = G_tmp

In [124]:
for k1 in adjDict:
    for k2 in adjDict[k1]:
        adjDict[k1][k2] = OrderedDict(sorted(adjDict[k1][k2].items(), key=lambda x: x[1]))

In [125]:
for key in adjDict:
    print(key, len(adjDict[key]))

1 18
6 36
24 71
1.2 125
1.4 125
1.6 125
1.8 125
2.0 125


In [126]:
with open(r'outData/graph-2.pkl', 'wb') as file:
    pickle.dump(adjDict, file)

In [127]:
type(adjDict[1.2][30])

collections.OrderedDict

In [128]:
adjDict[1.2][30][30]

0